# Question 3

The CRUD chemical plant produces as part of its production process a noxious compound called chemical X. Chemical X is highly toxic and needs to be disposed of properly. Fortunately, CRUD is linked by a pipe system to the FRESHAIR recycling plant that can safely reprocess chemical X. On any given day, the CRUD plant will produce the following amount of chemical X (the plant operates between 9am and 3pm only):

| Time                   | 9-10AM | 10-11AM | 11AM-12PM | 12-1PM | 1-2PM | 2-3PM |
|:----------------------:|:------:|:-------:|:---------:|:------:|:-----:|:-----:|
| Chemical X (in liters) | 300    | 240     |  600      |  200   |  300  |  600  |

Becuase of environmental regulation, at no point in time is the CRUD plant allowed to keep more than 1000 litres on site and no chemical X is allowed to be kept overnight. At the top of every hour, at most 650 litres of chemical X can be sent to the FRESHAIR recycling plant. The cost of recycling chemical X is different for every hour:

| Time                   | 10AM   | 11AM    | 12PM      | 1PM    | 2PM   | 3PM   |
|:----------------------:|:------:|:-------:|:---------:|:------:|:-----:|:-----:|
| Chemical X (in liters) | 30     |  40     |  35       |  45    |  38   |  5 0  |

You need to decide how much chemical to send from the CRUD plant to the FRESHAIR recycling plant at the top of each hour, so that you can minimize the total recycling cost but also meet all of the evironmental constraints. Formulate this problem as an LP.

In [1]:
# declaring model
using JuMP, HiGHS

chemical = Model(HiGHS.Optimizer)

A JuMP Model
Feasibility problem with:
Variables: 0
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: HiGHS

In [2]:
# vector for amount of chemical X produced each hour
production = [300 240 600 200 300 600]
# vector for cost of sending chemical X each hour 
cost = [30 40 35 45 38 50]

1×6 Matrix{Int64}:
 30  40  35  45  38  50

In [3]:
# declaring variable for current amount of chemical X at the end of each hour
@variable(chemical, 1000 >= current[1:6] >= 0)
# declaring variable for amount of chemical X sent to FRESHAIR
@variable(chemical, 650 >= sent[1:6] >= 0)

6-element Vector{VariableRef}:
 sent[1]
 sent[2]
 sent[3]
 sent[4]
 sent[5]
 sent[6]

In [4]:
# constraints 

# at the end of the final hour, amount of chemical X must be 0
@constraint(chemical, endcond, current[6] == 0)
# conservation
@constraint(chemical, flow[i in 1:6], production[i] - sent[i] == current[i])

6-element Vector{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.EqualTo{Float64}}, ScalarShape}}:
 flow[1] : -current[1] - sent[1] = -300.0
 flow[2] : -current[2] - sent[2] = -240.0
 flow[3] : -current[3] - sent[3] = -600.0
 flow[4] : -current[4] - sent[4] = -200.0
 flow[5] : -current[5] - sent[5] = -300.0
 flow[6] : -current[6] - sent[6] = -600.0

In [6]:
# defining objective function 
@objective(chemical, Min, sum(cost[i]*sent[i] for i in 1:6))

print(chemical)

In [7]:
# solving the model
optimize!(chemical);

# outputs detailed information about the solution process
@show solution_summary(chemical);

Presolving model
0 rows, 0 cols, 0 nonzeros
0 rows, 0 cols, 0 nonzeros
Presolve : Reductions: rows 0(-7); columns 0(-12); elements 0(-13) - Reduced to empty
Solving the original LP from the solution after postsolve
Model   status      : Optimal
Objective value     :  3.0000000000e+04
HiGHS run time      :          0.00
solution_summary(chemical) = * Solver : HiGHS

* Status
  Termination status : OPTIMAL
  Primal status      : FEASIBLE_POINT
  Dual status        : FEASIBLE_POINT
  Message from the solver:
  "kHighsModelStatusOptimal"

* Candidate solution
  Objective value      : 3.00000e+04
  Objective bound      : 0.00000e+00
  Relative gap         : Inf
  Dual objective value : 3.00000e+04

* Work counters
  Solve time (sec)   : 2.98404e-03
  Simplex iterations : 0
  Barrier iterations : 0
  Node count         : -1

